In [1]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 177.4 MB 41 kB/s 
  Created wheel for h2o: filename=h2o-3.38.0.2-py2.py3-none-any.whl size=177521195 sha256=e22a99fd635c0fb4d0031f35f86907672df02ebb4d0fbd921204993a8ba059b7
  Stored in directory: /root/.cache/pip/wheels/e4/ef/ab/a9b2e452e18b3dfea0b6114bc57c3b9e8b0e464eb2d03230e1
Successfully built h2o


In [4]:
import h2o
import pandas as pd
from h2o.estimators import H2ORandomForestEstimator

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpnhsy2dh_
  JVM stdout: /tmp/tmpnhsy2dh_/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpnhsy2dh_/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,17 days
H2O_cluster_name:,H2O_from_python_unknownUser_vvcpbw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [6]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HACKATHON NOV22/MODEL3 - H2O train_test_split+balance+validation/train_time_to_seconds.csv', index_col=0)
data.head()

,order_id,local_time,country_code,store_address,payment_status,n_of_products,products_total,final_status
0,33446280,51069,AR,55379,PAID,2,11.88,DeliveredStatus
1,33107339,42461,GT,23487,PAID,2,5.20,DeliveredStatus
2,32960645,42833,CR,62229,PAID,1,6.03,DeliveredStatus
3,32089564,72921,ES,29446,PAID,6,6.37,DeliveredStatus
4,32157739,77536,AR,13917,PAID,1,5.36,CanceledStatus


In [8]:
data['final_status']=pd.get_dummies(data['final_status'], drop_first=True)
data.head()

,order_id,local_time,country_code,store_address,payment_status,n_of_products,products_total,final_status
0,33446280,51069,AR,55379,PAID,2,11.88,1
1,33107339,42461,GT,23487,PAID,2,5.20,1
2,32960645,42833,CR,62229,PAID,1,6.03,1
3,32089564,72921,ES,29446,PAID,6,6.37,1
4,32157739,77536,AR,13917,PAID,1,5.36,0


In [9]:
hf = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
hf['final_status'] = hf['final_status'].asfactor()

In [11]:
# Parameters that are going to predict and the response
predictors = ['order_id', 'local_time', 'country_code', 'store_address', 'payment_status', 'n_of_products', 'products_total']
response = 'final_status'

In [13]:
train, valid = hf.split_frame(ratios=[0.8], seed=1)

In [19]:
model = H2ORandomForestEstimator(ntrees=10,
                                    max_depth=5,
                                    min_rows=10,
                                    calibrate_model=True,
                                    calibration_frame=valid,
                                    binomial_double_trees=True,
                                    balance_classes = True)

In [20]:
model.train(x=predictors,
               y=response,
               training_frame=train,
               validation_frame=valid)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: DRF_model_python_1668366237572_23


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    10                 20                          8380                   5            5            5             25            32            28.7

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.3910218405617545
RMSE: 0.6253173918593297
LogLoss: 1.1342380665161014
Mean Per-Class Error: 0.4364741986996873
AUC: 0.6368865131412587
AUCPR: 0.6083572632544821
Gini: 0.2737730262825173

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.8526263560383679
       0     1      Error    Rate
-----  ----  -----  -------  -----------------
0      7376  31140  0.8085   (31140.0/38516.0)
1      2479  35983  0.0645   (2479.0/38462.0)
Total  9855  67123  0.4367   (33619.0/76978.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.852626     0.681593  187
max f2                       0.768654     0.835557  266
max f0point5                 0.893854     0.603232  101
max accuracy                 0.897149     0.601977  95
max precision                1            1         0
max recall                   0.0740882    1         398
max specificity              1            1         0
max absolute_mcc             0.896619     0.204726  96
max min_per_class_accuracy   0.904031     0.599605  81
max mean_per_class_accuracy  0.897149     0.602005  95
max tns                      1            38516     0
max fns                      1            38461     0
max fps                      0            38516     399
max tps                      0.0740882    38462     398
max tnr                      1            1         0
max fnr                      1            0.999974  0
max fpr                      0            1         399
max tpr                      0.0740882    1         398

Gains/Lift Table: Avg response rate: 49.97 %, avg score: 87.83 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100232                   0.928177           1.43085   1.43085            0.715019         0.930599  0.715019                    0.930599            0.0143416       0.0143416                  43.0849   43.0849            0.00863204
2        0.0201364                   0.926453           1.3901    1.41038            0.694656         0.927128  0.704792                    0.928856            0.0140584       0.0284                     39.01     41.0383            0.0165179
3        0.0300437                   0.925342           1.3774    1.39951            0.688312         0.925789  0.699358                    0.927845            0.0136464       0.0420464                  37.7403   39.9508            0.0239918
4        0.0400026                   0.92471            1.35219   1.38773            0.675711         0.925048  0.693471                    0.927148            0.0134662       0.0555126                  35.2187   38.7727            0.0310026
5        0.05                        0.924206           1.23107   1.3564             0.615187         0.924484  0.677818                    0.92

In [21]:
# Model performance
performance = model.model_performance(test_data=valid)

print (performance)

ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.09654423850107731
RMSE: 0.31071568756835777
LogLoss: 0.3400384035587584
Mean Per-Class Error: 0.49311486279974515
AUC: 0.607490800649955
AUCPR: 0.9221096529552953
Gini: 0.21498160129990995

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7244662708377572
       0    1      Error    Rate
-----  ---  -----  -------  ----------------
0      18   1181   0.985    (1181.0/1199.0)
1      12   9648   0.0012   (12.0/9660.0)
Total  30   10829  0.1099   (1193.0/10859.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.724466     0.941774  378
max f2                       0.655504     0.975797  398
max f0point5                 0.808083     0.910915  327
max accuracy                 0.724466     0.890137  378
max precision                0.940685     1         0
max recal

In [22]:
model.F1()

[[0.8526263560383679, 0.6815930293128759]]

In [32]:
model.confusion_matrix()

,0,1,Error,Rate
0,7376.0,31140.0,0.8085,(31140.0/38516.0)
1,2479.0,35983.0,0.0645,(2479.0/38462.0)
Total,9855.0,67123.0,0.4367,(33619.0/76978.0)


In [23]:
metrics_rf = pd.DataFrame(model.F1(), columns=['threshold', 'f1'])
metrics_rf.to_csv('/content/drive/MyDrive/Colab Notebooks/HACKATHON NOV22/MODEL3 - H2O train_test_split+balance+validation/metrics_rf_raw.csv')

In [ ]:
# Let's import the data for the prediction

In [25]:
test_X = h2o.import_file('/content/drive/MyDrive/Colab Notebooks/HACKATHON NOV22/MODEL3 - H2O train_test_split+balance+validation/test_X_seconds.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [26]:
pred = model.predict(test_X)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [27]:
prediction_rf = pred.as_data_frame()

In [28]:
prediction_rf.to_csv('/content/drive/MyDrive/Colab Notebooks/HACKATHON NOV22/MODEL3 - H2O train_test_split+balance+validation/prediction_rf_raw.csv')